## Curso Python para Economistas
### Trabajo Práctico Nº 6 (integrador)

### Fecha de entrega:
Lunes 14/11* a las 17:00 hs

\* *tienen una semana más para resolver este TP*

### Modalidad de entrega y trabajo
- Este TP es **individual**.
- Un repositorio **privado** debe ser creado en GitHub para el TP, y dar acceso a los 5 profesores.
- Cuando hayan hecho ese último commit, copien la URL para clonar su repositorio y péguenla en
[este Google Sheet](https://docs.google.com/spreadsheets/d/1JmrvClRxxb3luzbDrzAMUC2cfQXyGIV0cev0JTi_Ff4/edit?usp=sharing), en la hoja del TP6. Al ser un repositorio privado, solo los colaboradores habilitados podrán clonarlo.
- Al finalizar el trabajo práctico deben hacer un último commit y push en su repositorio de GitHub con el mensaje `"Entrega final del TP6"`. Antes de la fecha y hora de entrega pueden hacer cuantos cambios quieran en el repositorio, pero luego de la hora de corte no deben hacer más cambios. Si un commit con el mensaje anterior se realiza luego de la hora de entrega, se supondrá que la entrega tardía fue intencional y se utilizarán los días de gracia. La última versión en el repositorio es la que será evaluada. Para esto es importante que no completen el Google Sheet hasta que no hayan finalizado el TP, como tampoco hacer pushes posteriores a la entrega.
- El estilo tendrá un peso en la nota de este trabajo. Pueden utilizar esta [guía de estilo](https://recursospython.com/pep8es.pdf) como referencia.

___

## Objetivo

El objetivo de este trabajo es integrar datos de anuncios de compraventa/alquiler de Argentina, provenientes del sitio www.properati.com, con datos censales de población de los partidos de la provincia de Buenos Aires (del año 2010, ya que los del último censo no están todavía disponibles); [ <br> 
Si les interesaría trabajar con otras provincias (en lugar de la provincia de Buenos Aires), **son bienvenidos a hacerlo**; pero, por simplicidad, las consignas se redactarán suponiendo que trabajan con partidos de la provincia de Buenos Aires.<br>

Este TP va a incluir los temas de la semana 6 (datos georreferenciados y mapas), pero al ser integrador también necesitarán revisar conceptos de semanas previas, en particular cómo operar con dataframe's de Pandas, cómo realizar gráficos con Matplotlib o Seaborn, y trabajar con widgets interactivos.

___

## Datos

En este trabajo vamos a utilizar dos nuevos conjuntos de datos, llamémoslos `propiedades` y `población_censo2010`. Además, vamos a trabajar con datos georreferenciados de provincias y departamentos/partidos, con los que ya trabajamos en la clase sincrónica 6.

#### `propiedades`
Es un archivo tabular CSV comprimido (extensión `.csv.gz`) que se pueden bajar de la página www.properati.com.ar/data. Este archivo contiene una series de registros (filas), donde cada uno corresponde a una propiedad que está en compra/venta o en alquiler en un dado período. La descripción de las columnas se puede consultar en el link anterior. Este es [el link](https://storage.googleapis.com/properati-data-public/ar_properties.csv.gz) para bajarse el archivo, que pesa 289MB. <br>

#### `población_censo2010`
Aquí deben bajarse la tabla P1-P de este link: https://www.indec.gob.ar/indec/web/Nivel4-CensoProvincia-3-6-06-000-2010. La misma contiene la población por partido en 2001, 2010 y la variación intercensal. Para este TP, sólo nos interesará la población en 2010. Si desean trabajar con otra provincia, pueden realizar la consulta correspondiente en https://www.indec.gob.ar/indec/web/Nivel4-Tema-2-41-135.

#### `departamentos_georef` y `provincias_georef`
Estos archivos, ya conocidos de la clase 6, contienen los datos georreferenciados de los departamentos y provincias de Argentina.


___

## Consignas

0. Bajar los datos de Properati correspondientes a Argentina, y la planilla XLS de población.

___

1. **Cargar y procesar los datos de propiedades (15 pts)** <br>
Leer los datos de propiedades (recordar que se trata de un CSV) en un `DataFrame` llamado `propiedades_df`. Notar que los datos están comprimidos (tiene extensión `.gz`). En lugar de descomprimir el archivo en el disco, usar el argumento `compression='gzip'` en la función de lectura.
    - Examinar el número de filas y columnas del DataFrame resultante.
    - Eliminar las columnas `id`, `ad_type`, `title`, `description`, `l1`, `l4`, `l5` y `l6`.
    - Usar la columna adecuada para filtrar los registros correspondientes a la provincia de Buenos Aires (u otra provincia de Argentina o CBA si así lo desean). Para esto puede ser útil el método `unique` el cual les permitirá examinar los valores únicos correspondientes a una columna. También puede resultar útil el método `isin`. Examinar nuevamente el número de filas y columnas del DataFrame resultante.
    - Filtrar los registros que tengan datos faltantes en las columnas de precio, superficie (total y cubierta), latitud y longitud. Para eso será util el método `dropna`, y para usarlo deben prestar atención a los argumentos `axis` y `subset`. Examinar nuevamente el número de filas y columnas del DataFrame resultante.    
    - Guardar el DataFrame como csv. Esto es sólo por conveniencia, para que no tengan que leer el archivo original y preprocesarlo desde cero si necesitan reiniciar el kernel de Jupyter. En el futuro pueden leer directamente el archivo final. Como referencia, este archivo (para la provincia de Buenos Aires) debería pesar unos 23 MB. No hace falta que lo incluyan en la entrega del trabajo.

In [ ]:
import pandas as pd

In [ ]:
PROPERATI_ORIGINAL = "" # ruta al archivo original
ARCHIVO_PROPIEDADES_FILTRADO = "" # ruta al archivo final

2. **Analicemos un poco el DataFrame. (10 pts)** <br>
Por inspección del DataFrame (y tal vez por experiencia) sabemos que los alquileres de inmuebles se suelen realizar en pesos argentinos (ARS); y las compraventas, en dólares (USD). Generar un DataFrame agregado que cuantifique cuántas operaciones de cada tipo se realizan en ARS y cuántas en USD, de acuerdo al tipo de propiedad. La estructura del DataFrame debe ser la siguiente: el tipo de operación y la unidad monetaria (`ARS` o `USD`) deben formar parte de un multiíndice. Los distintos tipos de propiedades deben ser columnas. Los valores deben ser las cuentas (cuántos registros hay para la dada combinación). 
Este link puede resultar útil para saber qué operación de reformateo realizar: https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html


In [ ]:
propiedades_df = pd.read_csv(ARCHIVO_PROPIEDADES_FILTRADO)

3. **Convertir el DataFrame de Properati en GeoDataFrame. (13 pts)** <br>
El DataFrame generado tiene dos columnas para las coordenadas de latitud y longitud del inmueble. En este ejercicio, lo convertiremos en un GeoDataFrame.
- Utilizando el método `apply`,  generar una Series cuyo contenido sean puntos de `shapely` (es decir, objetos geométricos), cada uno construido a partir de la latitud y longitud del inmueble. Para eso, definir una función auxiliar llamada `generar_geometria`, que reciba una fila del DataFrame y devuelva un `Point` (prestar atención al orden de la longitud y la latitud). Agregar esta Series como una nueva columna llamada `geometry`. Eliminar las columnas de latitud y longitud.
- Generar un GeoDataFrame a partir del DataFrame anterior. Esto se puede hacer llamando al constructor de GeoDataFrame usando el argumento `geometry` al que se le pasa el nombre de la columna que se va a usar como la geometría asociada a cada registro.

In [ ]:
from shapely.geometry import Point
import geopandas as gpd

Ejemplo de cómo inicializar un punto:

In [ ]:
latitud, longitud = -34.93, -57.95
la_plata = Point(longitud, latitud)
la_plata

In [ ]:
def generar_geometria(fila):                
    # Completar
    return 

4. **Generar un GeoDataFrame con los partidos de Buenos Aires (7pts)**

Usando Geopandas, cargar los shapefiles de departamentos/partidos y provincias en sendos geodataframes. Repetir el procedimiento seguido en la clase sincrónica 6 para asociar los departamentos/partidos a sus respectivas provincias. Filtrar el GeoDataFrame resultante de manera de quedarse con los registros correspondientes a sólo la provincia de Buenos Aires (o la provincia que hayan elegido). Llamarlo `partidos_bsas_gdf` o `departamentos_<provincia>_gdf`.

In [ ]:
PROVINCIAS = "" # ruta al shapefile de provincias
DEPARTAMENTOS = "" # ruta al shapefile de departamentos

5. **Asociar inmuebles a sus respectivos partidos mediante información georreferenciada (10pts)** <br>
Notar que en el GeoDataFrame de Properati tenemos la columna `l3`, pero no es claro si esto representa una localidad o un partido. 
En este ejercicio, vamos a realizar un join espacial (`sjoin`) de `propiedades_gdf` con el `partidos_bsas_gdf` generado en el punto anterior. Luego, deben determinar para cuántos de los registros, la columna `l3` coincide con el nombre del partido proveniente de `partidos_bsas_gdf`. Dejar mencionado como comentario algunos casos que detecten en los que no coinciden estos valores. <br>
Finalmente, descartar los departamentos con menos de 10 registros (recordar `groupby(...).filter` de la clase 4).

6. **Leer los datos de población por partido. (5pts)** <br>
Deben cargar la información de población en un DataFrame usando Pandas, y producir un DataFrame final que contenga dos columnas: el nombre del partido y la población en 2010. Se permiten manipulaciones manuales del archivo XLS bajado de la página de INDEC. Deben incluir el archivo XLS "curado" en el repositorio al momento de la entrega, para que los profesores puedan reproducir los resultados ejecutando el código del notebook.



7. **Número de anuncios por partido, per cápita (20 pts)**. <br>
Generar un mapa de la provincia de Buenos Aires, coloreando cada partido de acuerdo al número de registros normalizadp por la población del mismo (en 2010). Utilizar un widget reactivo para visualizar el mapa de acuerdo al tipo de operación: venta, alquiler o alquiler temporal. _Se evaluará la presentación del mapa_.


8. **Dos gráficos a elección (20 pts + bonus si usan datos adicionales)**. <br>
Usando los datos que hemos procesado, hacer dos gráficos que permitan visualizar algunas relaciones que les parezcan interesantes. Dejamos la elección de las mismas a criterio de ustedes, así como la de los tipos de gráfico a utilizar. Uno de los gráficos puede ser nuevamente un mapa, pero debe ser suficientemente distinto al del punto anterior. Se admite usar bases de datos adicionales si así lo desean (esto les dará puntos extra!) <br>Noten que si examinan alguna relación que tenga que ver con precios, deben tener en cuenta la unidad monetaria en la que están expresados los mismos (pueden simplemente descartar la minoritaria, por ejemplo compraventas en ARS). También pueden utilizar widgets. _Se evaluará la presentación de los gráficos/mapas_.